# EXTRA CREDIT HOMEWORK 

In [22]:
from __future__ import print_function, division
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import multivariate_normal
import seaborn as sns; sns.set()
from demo import fairness_demo
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_score, recall_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

# Problem 1  [25 points]
Consider the four datasets returned from the function get_dataset($d$) for $d=2,3,4, \& \;5$. Here $d$ is the dimensionality of the non-sensitive covariates, which are returned in the matrix $X$, whereas the vectors $y$ and $x_{\rm{sensitive}}$ store the target labels and sensitive covariate, respectively. (As such the structure of the data is exactly analagous to what we have in the fairness demo notebook from the lecture.)

In [ ]:
def get_gaussian_data(mean, cov, class_label, n_samples):
    nv = multivariate_normal(mean = mean, cov = cov)
    X = nv.rvs(n_samples)
    y = np.ones(n_samples, dtype=float) * class_label
    return X,y

### this function returns the dataset
### e.g. X, y, x_sensitive = get_dataset(2)

def get_dataset(d):
    np.random.seed(5)
    mu1 = 0.5*np.ones(d)
    mu2 = -0.5*np.ones(d)
    sigma1 = np.eye(d)
    sigma2 = np.eye(d)
    X1, y1 = get_gaussian_data(mu1, sigma1, 1, 10000*d) # positive class
    X2, y2 = get_gaussian_data(mu2, sigma2, -1, 10000*d) # negative class

    X = np.vstack((X1, X2)) # non-sensitive covariates
    y = np.hstack((y1, y2)) # class labels
    x_sensitive = np.ones(X.shape[0])
    x_sensitive[X[:,0]<0.0] = 0 # sensitive covariate; 
                                # 0 is the protected class, 1 is the non-protected class
    return X, y, x_sensitive

# part a)  [10 points]
Using the logistic regression classifier provided by the python class fairness_demo (just like in the lecture notebook) calculate the accuracy and p%-rule ratio for all four datasets using the unconstrained classifier (i.e. no fairness constraints are imposed).

# part b) [10 points]
Note that for all four datasets the "four-fifths rule" is very much not satisfied. For each dataset impose the minimum fairness constraint such that the four-fiths rule is satisfied. What is the loss in accuracy for each dataset as compared to the unconstrained classifier performance?

# part c) [5 points]
Notice that (as least as far as the four datasets for $d=2,3,4, \& \;5$ are concerned) as the dimension $d$ increases  the following things happen:

- the accuracy increases
- the p%-rule ratio for the unconstrained classifier increases
- the accuracy losses as calculated in part b decrease (at least approximately up to fluctuations)

Look at the function get_dataset($d$) and consider how the generated dataset changes as a function of $d$. Do you expect the behavior described above to continue for all values of $d>5$? If so, explain why. If not, explain why not.
<br><br>

# Problem 2  [10 points]

Read the following review by Barocas and Selbst (or as much of the review as you find interesting):

http://papers.ssrn.com/sol3/papers.cfm?abstract_id=2477899

Answer the following questions on the basis of what you've read.

# part a)

Consider the logic underlying the "fairness aware" classifier we explored in the previous problem. Consider the principle of nondiscrimination versus the principle of antisubordination. Which (if any) of the two principles is more in line with the approach taken by the algorithm? Why?

# part b)

Consider the "fairness aware" classifier we explored in the previous problem. In itself does it offer a solution to the problem of "masking" as described in the review?
<br><br>

# Problem 3  [5 points]

Consider the following illustration of a dataset in which the positive target labels are marked with plus signs, the green points constitute the non-protected class, and the blue points constitute the protected class. The distribution of the non-protected class is illustrated on the left, the distribution of the protected class is illustrated in the middle, and the graphic on the right shows the combined dataset.

<img src="dataset.png">

Consider applying the "fairness aware" classifier in Problem 1 to the combined dataset, imposing fairness constraints such that the four-fifths rule is satisfied. Do you expect the loss of accuracy as you go from the unconstrained to the constrained classifier to be large or small? Why?


<br><br>

# Problem 4  [15 points]
#### (This is question 2.2 from Dunning's book)

In a study of the effect of police presence on the incidence of crime, Di Tella and Schargrodsky (2004) write:

“Following a terrorist attack on the main Jewish center in Buenos Aires, Argentina, in July
1994, all Jewish institutions received police protection… Because the geographical
distribution of these institutions can be presumed to be exogenous in a crime regression, this hideous event constitutes a natural experiment.”

The authors find that blocks which were allocated extra police forces due to the presence of a Jewish institution experienced lower motor vehicle theft rates.  The control group consists of blocks in the same neighborhoods that do not have Jewish institutions.

Answer the following three questions __in at least 6-10 sentences__.

### part a) 

What do the authors mean by “presumed exogenous in a crime regression” and what is the relationship to as-if random assignment?  
### part b) 
What are some potential threats to as-if random assignment?  [give at least two examples of potential threats]
### part c) 
How might these threats be evaluated empirically?
<br><br><br><br>

### a)

That the geographical distribution of the Jewish instiutions is "presumed exogenous in a crime regression" means that they presume the locatons do not have a causal relationship with crime such as motor theft.  Since they presume that there is no causal relationship between the locations and crime, then the groups near a Jewish instituion can be separated from those not near one as a "random" treatment and control group with "as-if random assignment."

### b)

* There may be factors that the authors didn't consider which are not evenly distributed between the two groups.
* The treatment and control groups are formed post-hoc and the experiment is formed given that, instead of forming an experiment and then creating a randomized trial.  Because there is an outside force which created these two groups, there could be other external factors affecting the results.  For example, if the above experiment is supposed to test the effect of police force on preventing motor theft, it may be that the communities themselves are just on higher alert due to the terror attack, and not the police force, that is reducing theft

### c)

* This can be evaluated by increasing the number of features being considering and seeing if there is an unequal distribution between the two groups.
* Look for either Jewish areas which didn't recieve an increased police presenence, or places with increased police presence for another reason and see if either see similar impacts.

# Problem 5  [45 points]
Consider the Titanic dataset below

In [2]:
data=pd.read_csv("https://serv.cusp.nyu.edu/classes/ML_2016_Spring/Bonus/titanic3.csv").dropna(subset=['age'])

target = data['survived']
X = data[['age', 'sex', 'pclass', 'sibsp', 'parch']]
X = pd.get_dummies(X)
X_train, X_test, target_train, target_test = train_test_split(
    X, target, test_size=0.25, random_state=1
)

In [3]:
def report(pred, target_test):
    print("Classification accuracy for test set =", 1.0*sum(target_test==pred)/len(pred))
    
    print("Precision (Survived):", precision_score(target_test, pred))
    print("Precision (Didn't Survive):", precision_score(target_test, pred, pos_label=0))
    
    print("Recall (Survived):", recall_score(target_test, pred))
    print("Recall (Didn't Survive):", recall_score(target_test, pred, pos_label=0))
    

## Data dictionary
NAME: titanic3<br>
SIZE: 1309 Passengers, 14 Variables<br><br>

VARIABLE DESCRIPTIONS<br>
Pclass: Passenger Class (1 = 1st; 2 = 2nd; 3 = 3rd) <br>
survival: Survival (0 = No; 1 = Yes)<br>
name: Name<br>
sex: Sex<br>
age: Age<br>
sibsp: Number of Siblings/Spouses Aboard<br>
parch: Number of Parents/Children Aboard<br>
ticket: Ticket Number<br>
fare: Passenger Fare (British pound)<br>
cabin: Cabin<br>
embarked: Port of Embarkation (C = Cherbourg; Q = Queenstown; S = Southampton)<br>
boat: Lifeboat<br>
body: Body Identification Number<br>
home.dest: Home/Destination

# part a) [30 points]

Your goal is to train a classifier for the binary attribute “survival" using age, sex, pclass, sibsp, and parch as features. You will do so using three different machine learning techniques:

i) Naive Bayes Classification. [10 points]

ii) Support Vector Machine. Try a linear SVM with soft margins as well as kernel SVM with polynomial and Gaussian kernels. Make sure to use a validation set to choose hyperparameters for each model where applicable. [10 points]

iii) Random Forest Classification. [10 points]

For each of the three models report out-of-sample accuracy--in order to do so, you will of course need to split the dataset into a training dataset and a test dataset.

# part b)  [15 points]

Repeat the exercise in part a, this time using cross validation. Report the mean accuracy for each model after doing 10 random splits of the data into train and test sets.

## i) Naive Bayes Classification 

In [4]:
gnb = GaussianNB()
gnb.fit(X_train, target_train)
pred = gnb.predict(X_test)

report(pred, target_test)

Classification accuracy for test set = 0.824427480916
Precision (Survived): 0.782178217822
Precision (Didn't Survive): 0.850931677019
Recall (Survived): 0.766990291262
Recall (Didn't Survive): 0.861635220126


## ii) Support Vector Machine

### Linear

In [5]:
svc = SVC(kernel='linear')
svc.fit(X_train, target_train)
pred = svc.predict(X_test)

report(pred, target_test)

Classification accuracy for test set = 0.812977099237
Precision (Survived): 0.759615384615
Precision (Didn't Survive): 0.848101265823
Recall (Survived): 0.766990291262
Recall (Didn't Survive): 0.842767295597


### Poly

In [19]:
svc = SVC(kernel='poly', max_iter=-1, tol=.01, degree=2)
svc.fit(X_train, target_train)
pred = svc.predict(X_test)

report(pred, target_test)

Classification accuracy for test set = 0.843511450382
Precision (Survived): 0.787037037037
Precision (Didn't Survive): 0.883116883117
Recall (Survived): 0.825242718447
Recall (Didn't Survive): 0.85534591195


### RBF

In [6]:
svc = SVC(kernel='rbf')
svc.fit(X_train, target_train)
pred = svc.predict(X_test)

report(pred, target_test)

Classification accuracy for test set = 0.835877862595
Precision (Survived): 0.794117647059
Precision (Didn't Survive): 0.8625
Recall (Survived): 0.78640776699
Recall (Didn't Survive): 0.867924528302


## iii) Random Forest Classification

In [25]:
clf = RandomForestClassifier(n_jobs=-1, n_estimators=500)
clf = clf.fit(X_train, target_train)
pred = clf.predict(X_test)

report(pred, target_test)

Classification accuracy for test set = 0.793893129771
Precision (Survived): 0.702479338843
Precision (Didn't Survive): 0.872340425532
Recall (Survived): 0.825242718447
Recall (Didn't Survive): 0.77358490566


# B

In [40]:
gnb = GaussianNB()
for clf, name in [
    (GaussianNB(), 'Naive Bayes'), 
    (SVC(kernel='linear'), 'Linear SVM'),
    (SVC(kernel='rbf'), 'RBF SVM'),
    (SVC(kernel='poly', tol=.1, degree=2), 'Poly SVM'),
    (RandomForestClassifier(n_jobs=-1, n_estimators=300), 'Random Forest')
]:
    scores = cross_val_score(clf, X, target, cv=10, scoring='accuracy')
    print(name, "- Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Naive Bayes - Accuracy: 0.78 (+/- 0.15)
Linear SVM - Accuracy: 0.78 (+/- 0.14)
RBF SVM - Accuracy: 0.78 (+/- 0.17)
Poly SVM - Accuracy: 0.80 (+/- 0.15)
Random Forest - Accuracy: 0.73 (+/- 0.18)
